# CCMC ROR satellite extraction wrapper

In [ ]:
import kamodo
from kamodo_ccmc.readers.sat_extractions import SATEXTRACT

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.io as pio
import plotly.graph_objects as go

runID     = '6687'
coord     = 'GSE'
satellite = 'GOES-12'

ror = SATEXTRACT(runID, coord, satellite)
ror

In [ ]:
fig=ror.get_plot(type="1Dpos", scale="R_E")
iplot(fig)

The above plot may be generated by calling kamodo's automated plotting routines.

In [ ]:
fig = ror.plot('X__GSE', 'Y__GSE', 'Z__GSE')
fig

The same technique may be used to plot a custom time sequence.

In [ ]:
from datetime import timedelta

In [ ]:
# generate times by offsetting first 2000 points by 5.3 seconds
t_custom = ror.dtarray[:2000] + timedelta(seconds=5.3) 
ror.plot(X__GSE=dict(t=t_custom), Y__GSE=dict(t=t_custom), Z__GSE=dict(t=t_custom))

In [ ]:
ror

In [ ]:
fig=ror.get_plot(type="1Dvar", var="B_z")
iplot(fig)

Similarly, we can reproduce the above plot of `B_z`:

In [ ]:
fig=ror.plot('B_z__GSE')
fig

In [ ]:
# Combine multiple values in one plot, and rename y-axis
fig=ror.get_plot(type="1Dvar", var="B_x")
figA=ror.get_plot(type="1Dvar", var="B_y")
fig.add_trace(figA.data[0])
figB=ror.get_plot(type="1Dvar", var="B_z")
fig.add_trace(figB.data[0])
fig.update_yaxes(title_text='nT')
iplot(fig)

In [ ]:
ror.plot('B_x__GSE', 'B_y__GSE', 'B_z__GSE')

In [ ]:
ror

### Read the same satellite and dates from CDAWeb

In [ ]:
from kamodo_ccmc.readers.hapi import HAPI

server = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset = 'GOES12_K0_MAG'
parameters = 'B_GSE_c, SC_pos_se'
start      = ror.start
stop       = ror.stop

hapi2 = HAPI(server, dataset, parameters, start, stop)

In [ ]:
hapi2

In [ ]:
hapi2.detail()

In [ ]:
import numpy as np

In [ ]:

hapi2.zhat(np.array([[3,4,5]]))

In [ ]:
hapi2.detail()

In [ ]:
hapi2.plot('SC_pos_se')

In [ ]:
hapi2.plot('SC_x__pos_se', 'SC_y__pos_se', 'SC_z__pos_se')

In [ ]:
fig=hapi2.get_plot(coord='GSE', type="1Dpos", scale="km")
iplot(fig)

In [ ]:
fig=hapi2.get_plot(type="1Dvar", var="B_GSE_c")
iplot(fig)

In [ ]:
hapi2.plot('B_x__GSE_c', 'B_y__GSE_c', 'B_z__GSE_c')

### Compare the B_y values from the extraction to the observed values

In [ ]:
fig=ror.get_plot(type="1Dvar", var="B_x")
figA=ror.get_plot(type="1Dvar", var="B_y")
fig.add_trace(figA.data[0])
figB=ror.get_plot(type="1Dvar", var="B_z")
fig.add_trace(figB.data[0])
figC=hapi2.get_plot(type="1Dvar", var="B_GSE_c")
fig.add_trace(figC.data[0])
fig.add_trace(figC.data[1])
fig.add_trace(figC.data[2])
fig.update_yaxes(title_text='nT')
iplot(fig)

In [ ]:
hapi2

In [ ]:
ror

In [ ]:
ror['B_x__GSE_c'] = hapi2.B_x__GSE_c

In [ ]:
ror

In [ ]:
from kamodo import Kamodo

In [ ]:
k = Kamodo(B_x__GSE=ror.B_x__GSE,
           B_x__GSE_c=hapi2.B_x__GSE_c)
k

In [ ]:
k.plot('B_x__GSE', 'B_x__GSE_c' )

# Interpolation

In [ ]:
from datetime import datetime,timezone
# datetime
dt = ror.dtarray[0]
print(dt)
# datetime to timestamp
ts = dt.timestamp()
print(ts)
#timestamp back to datetime
dt2 = datetime.fromtimestamp(ts, tz=timezone.utc)
print(dt2)
print(dt2.strftime("%Y-%m-%dT%H:%M:%SZ"))

In [ ]:
# Reference points at start of arrays
print(ror.dtarray[0],'\n',ror.tsarray[0],'\n',ror.variables['X']['data'][0])

In [ ]:
# Interpolate one point
ts=ror.dtarray[0].timestamp()+10.
print(ts)
print(ror.X__GSE(ts))

In [ ]:
# Interpolate array of points (method 1)
tss=(ror.dtarray[0].timestamp()+10.,
     ror.dtarray[0].timestamp()+20.)
print(ror.X__GSE(list(tss)))

In [ ]:
# Interpolate array of points (method 2 --preferred)
var='X'
units=ror.variables[var]['units']
print(units)
tss=(ror.dtarray[0].timestamp()+10.,
     ror.dtarray[0].timestamp()+20.)
test = ror.variables[var]['interpolator'](tss)
print(test)